# Make parameter sweep submission 

Using new job quota on HPC of 50 `qsub`'s, each running 10000 jobs 

Author: Juvid Aryaman

Date: 19/03/18

In [1]:
!pwd

/home/juvid/Dropbox/Work/Mit_and_Metabolism/Networks_mtDNA_dynamics/Stochastic_sim/AJHG_feedback_controls/Param_sweeps/X_gen_linear_feedback_ctrl/Stoch_sim


In [2]:
import numpy as np
import pandas as pd
import pdb

%reload_ext autoreload
%autoreload 2

## Get steady states from ODE integration

In [3]:
dcols = ['param_index','ws','wf','ms','mf']
ss_vals = pd.DataFrame(columns=dcols)
for i in range(121):
    d = pd.read_csv('../Find_SS/Data/ode_sol_{}.csv'.format(i))
    ss_vals = ss_vals.append(d.iloc[-1,:][dcols[1:]].set_value('param_index',i))
    

In [4]:
ss_vals['param_index'] = ss_vals['param_index'].astype(int)
ss_vals.reset_index(inplace=True, drop=True)
ss_vals.head()

,param_index,ws,wf,ms,mf
0,0,237.790450,31.833246,101.910193,13.642820
1,1,244.924886,34.435589,104.967808,14.758110
2,2,264.082932,42.194695,113.178399,18.083441
3,3,321.944257,73.186670,137.976110,31.365716
4,4,447.985338,255.832485,191.993716,109.642494


In [5]:
ss_vals[['ws','wf','ms','mf']].sum(axis=1).min(),ss_vals[['ws','wf','ms','mf']].sum(axis=1).max()

(107.74637993619167, 2350.1598382655998)

## Make Submission

In [6]:
xi = 0.0
beta = 33.12
gamma = 0.03785142857142858
b = 1.2416523075924095e-05
kappa = 11.662903457629223
mu = 0.023
delta1 = 0.8
delta2 = 1.0
delta3 = 0.2
delta4 = 0.3

nominal_param = [xi,beta,gamma,b,kappa,mu,delta1,delta2,delta3,delta4]

In [7]:
work_dir = '/work/ja1109/networks/sing_birth_fus/X_general_ctrl'

In [8]:
c_file_name = 'X_gen_lin_feedback.c'
hpc_job_name = 'X_SW'

Define magnitudes and ratios for ($\beta, \gamma$). Let $M$ be magnitude and $R$ be ratio, then $\beta'=\beta M$ and $\gamma' = \beta' R_0 R$ where $R_0 = \gamma/\beta$. Then this implies $\gamma'=\gamma M R$.

In [9]:
n_points = 11
lg_ls_low_churn = -2
lg_ls_high_churn = 2

lg_ls_low_rat = -2
lg_ls_high_rat = 2

mags = 10**np.linspace(lg_ls_low_churn,lg_ls_high_churn,n_points) # magnitudes of the fusion/fission rate
rats = 10**np.linspace(lg_ls_low_rat,lg_ls_high_rat,n_points) # ratios of the fusion/fission rate

np.log10(mags)

array([-2. , -1.6, -1.2, -0.8, -0.4,  0. ,  0.4,  0.8,  1.2,  1.6,  2. ])

In [10]:
rat_nominal = gamma/beta

In [11]:
n_iters_per_block = 1000 # multiply by NREPEATS in the .c script for the total number of iterations
n_job_blocks = 20 # split the work across this many qsubs

In [12]:
initial_seed = np.round(np.random.uniform()*1e6).astype(int)
print(initial_seed)

435591


Create first set of parameter sweeps over ($\beta$,$\gamma$) at constant ($\kappa$,$b$).

In [13]:
h_target = 0.3

In [14]:
time_per_job = '24:00:00' 

Submission protocol

On local machine

`$python Make_submission_param_sweep.py` 

(do this locally because some work must be done to find the right settings)

`$rsyncr * hpc:$PBS_O_WORKDIR`

where `$PBS_O_WORKDIR` is the directory where you run the script from.

`$gcc -o3 closed_loop_deg_single.c -lm -o closed_loop_deg_single.ce`

`$bash master.sh`

In [15]:
def make_submission_string(initial_seed, param_block, n_iters_per_block, seed_inc, ics, new_param):
    submission_string = '{0} {1} {2} {3}'.format(initial_seed,param_block,n_iters_per_block,seed_inc)     
    
    for ic in ics:
        submission_string+=' '+str(ic)

    for p in new_param:
        submission_string+=' {}'.format(p)
    
    submission_string+='\n'

    return submission_string

In [16]:
def make_submission_script(job_block,start_row,stop_row,time_per_job, c_file_name, hpc_job_name):
    f = open('submission_{}.pbs'.format(job_block),'w')
    f.write('#!/bin/sh\n')
    f.write('#PBS -J {}-{}\n'.format(start_row, stop_row))
    f.write('#PBS -N {0}_{1}\n'.format(hpc_job_name, job_block))
    f.write('#PBS -l walltime={}\n'.format(time_per_job))
    f.write('#PBS -l select=1:ncpus=1:mem=1gb\n')
    
    f.write('ARGS=$(head -$PBS_ARRAY_INDEX $PBS_O_WORKDIR/comm | tail -1 )\n')
    f.write('OUTDIRVAL=$(head -$PBS_ARRAY_INDEX $PBS_O_WORKDIR/comm_dir | tail -1 )\n')
    f.write('printf -v DATA_DIR "p%d" $OUTDIRVAL\n')
    
    f.write('cp $PBS_O_WORKDIR/{}e $TMPDIR\n'.format(c_file_name))
    f.write('./{}e $ARGS\n'.format(c_file_name))
    f.write('cp output* $PBS_O_WORKDIR/$DATA_DIR\n')
    f.write('sleep 10\n')
    f.write('rm $PBS_O_WORKDIR/{}*\n'.format(hpc_job_name))

    f.close()

In [17]:
param_cols = ['xi','beta','gamma','b','kappa','mu','delta1','delta2','delta3','delta4', 
              'net_mag','net_rat','h_init','n_init',
              'ws_init','ms_init','wf_init','mf_init']

In [18]:
param_df = pd.DataFrame(columns=param_cols)

In [19]:
master_script = open('master.sh','w')
master_script.write('gcc -Wall -o3 {0} -lm -o {0}e\n'.format(c_file_name))

comm_script = open('comm','w')
comm_dir_script = open('comm_dir','w')

submn_script_counter = 0
job_counter = 0
max_jobs_per_submission = 10000

final_row = n_points*n_points*n_iters_per_block - 1

for i, mag in enumerate(mags):
    for j, ratio in enumerate(rats):
        
        # Figure out which job block I'm in
        if job_counter % max_jobs_per_submission == 0:
            start_row = submn_script_counter*max_jobs_per_submission
            stop_row = (submn_script_counter+1)*max_jobs_per_submission - 1
            if stop_row < final_row:
                make_submission_script(submn_script_counter,start_row+1,stop_row+1,time_per_job,c_file_name, hpc_job_name)
            else:
                make_submission_script(submn_script_counter,start_row+1,final_row+1,time_per_job,c_file_name, hpc_job_name)
            submn_script_counter += 1                      
        
        param_block = len(rats)*i+j
        
        newbeta = beta * mag # beta'
        newgamma = newbeta * rat_nominal * ratio # gamma'
        
        new_param = [k for k in nominal_param]
        new_param[1] = newbeta
        new_param[2] = newgamma
        
        ics = ss_vals[ss_vals['param_index']==param_block][['ws','ms','wf','mf']].round().astype(int).as_matrix()[0]
        
        if np.any(ics < 0):
            raise Exception('Bad ICs!')
                   
        n = np.sum(ics)
        h = (ics[1]+ics[3])/float(n)
        
        param_df = param_df.append(pd.DataFrame([np.hstack((new_param,(mag,ratio,h,n),ics))],columns=param_cols))
        
        # Comm
        for k in range(n_iters_per_block):
            s = make_submission_string(initial_seed, param_block, n_iters_per_block, k, ics, new_param)
            comm_script.write(s)
            comm_dir_script.write(str(param_block)+'\n')
            job_counter += 1
        
        master_script.write('echo "{}"\n'.format(param_block))
        master_script.write('mkdir -p {0}/p{1}\n'.format(work_dir,param_block))
        
for i in range(submn_script_counter):
    master_script.write('sleep 0.5\n')
    master_script.write("qsub submission_{}.pbs\n".format(i))
            
master_script.close()
comm_script.close()
comm_dir_script.close()
param_df.reset_index(inplace=True, drop=True)        

In [20]:
param_df.to_csv('param_sweep_vals.csv',index=False)

In [21]:
param_df

,xi,beta,gamma,b,kappa,mu,delta1,delta2,delta3,delta4,net_mag,net_rat,h_init,n_init,ws_init,ms_init,wf_init,mf_init
0,0.0,0.331200,0.000004,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,0.010000,0.300518,386.0,238.0,102.0,32.0,14.0
1,0.0,0.331200,0.000010,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,0.025119,0.300752,399.0,245.0,105.0,34.0,15.0
2,0.0,0.331200,0.000024,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,0.063096,0.299771,437.0,264.0,113.0,42.0,18.0
3,0.0,0.331200,0.000060,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,0.158489,0.299645,564.0,322.0,138.0,73.0,31.0
4,0.0,0.331200,0.000151,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,0.398107,0.300199,1006.0,448.0,192.0,256.0,110.0
5,0.0,0.331200,0.000379,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,1.000000,0.299883,1704.0,392.0,168.0,801.0,343.0
6,0.0,0.331200,0.000951,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,2.511886,0.300000,2090.0,207.0,89.0,1256.0,538.0
7,0.0,0.331200,0.002388,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,6.309573,0.300000,2250.0,91.0,39.0,1484.0,636.0
8,0.0,0.331200,0.005999,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,15.848932,0.299914,2314.0,38.0,16.0,1582.0,678.0
9,0.0,0.331200,0.015069,0.000012,11.662903,0.023,0.8,1.0,0.2,0.3,0.010000,39.810717,0.300000,2340.0,15.0,7.0,1623.0,695.0


In [22]:
!pwd

/home/juvid/Dropbox/Work/Mit_and_Metabolism/Networks_mtDNA_dynamics/Stochastic_sim/AJHG_feedback_controls/Param_sweeps/X_gen_linear_feedback_ctrl/Stoch_sim
